In [1]:
!pip install booknlp
!wget https://www.gutenberg.org/files/208/208.txt -O 208_daisy_miller.txt
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
wget: /shared/centos7/anaconda3/2021.05/lib/libuuid.so.1: no version information available (required by wget)
--2024-02-02 20:06:43--  https://www.gutenberg.org/files/208/208.txt
Connecting to 10.99.0.130:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 144615 (141K) [text/plain]
Saving to: ‘208_daisy_miller.txt’

100%[======================================>] 144,615     --.-K/s   in 0.08s   

2024-02-02 20:06:44 (1.72 MB/s) - ‘208_daisy_miller.txt’ saved [144615/144615]

^C
Traceback (most recent call last):
  File "/shared/centos7/anaconda3/2021.05/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/shared/centos7/anaconda3/2021.05/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/pip/__main__.py", line 21, in <module>
    from

In [2]:
import pandas as pd
from booknlp.booknlp import BookNLP
import csv
import json
from collections import Counter
import spacy

using device cpu


In [3]:
model_params={
		"pipeline":"entity,quote,supersense,event,coref",
		"model":"big",
	}

booknlp=BookNLP("en", model_params)

{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
--- startup: 9.837 seconds ---


In [4]:
df = pd.read_csv("mostrcent_gofundme_slim.csv",header = None, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
df2 = pd.read_csv('mostrcent_gofundme_slim.csv')
df2['desc']


0         On Saturday April 29th 2023. My brother in law...
1         Hello I'm Katherine, and I need help with my r...
2         Hi, I'm helping my cousin Jannah A. raise fund...
3         Hi, my friend and I have been filming car cont...
4         Two and a half years ago, our dear friends She...
                                ...                        
157863    I have been admitted in the ICU for sometime n...
157864    We are heartbroken by the loss of our father, ...
157865    Hi, Our names are Kimia, Savannah, and Teresa ...
157866    My husband, Bob is 72 yrs. old and disabled. H...
157867    Tim was a beloved colleague and friend, and ma...
Name: desc, Length: 157868, dtype: object

In [5]:
def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data
nlp = spacy.load('en_core_web_sm')
nlp.max_length
def get_input(input_file):
  inputFile= input_file
  outputDir='outputdirnew-original3'
  idd="output"

  booknlp.process(inputFile, outputDir, idd)

In [6]:
data=proc("outputdir/output.book")
with open ("outputdir/output.book", "r") as f:
    book_data = json.load(f)
book_data.keys()
book_data['characters'][0]['g']

{'inference': {'he/him/his': 0.371,
  'she/her': 0.379,
  'they/them/their': 0.245,
  'xe/xem/xyr/xir': 0.002,
  'ze/zem/zir/hir': 0.004},
 'argmax': 'she/her',
 'max': 0.379,
 'total': 2529.532}

In [10]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter
def create_character_data(data, printTop):
    character_data = {}
    for character in data["characters"]:

        agentList=character["agent"]
        patientList=character["patient"]
        possList=character["poss"]
        modList=character["mod"]

        character_id=character["id"]
        count=character["count"]

        referential_gender_distribution=referential_gender_prediction="unknown"

        if character["g"] is not None and character["g"] != "unknown":
            referential_gender_distribution=character["g"]["inference"]
            referential_gender=character["g"]["argmax"]

        mentions=character["mentions"]
        proper_mentions=mentions["proper"]
        max_proper_mention=""

        #Let's create some empty lists that we can append to.
        poss_items = []
        agent_items = []
        patient_items = []
        mod_items = []

        # just print out information about named characters
        if len(mentions["proper"]) > 0:
            max_proper_mention=mentions["proper"][0]["n"]
            for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
                poss_items.append((v,k))

            for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
                agent_items.append((v,k))

            for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
                patient_items.append((v,k))

            for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
                mod_items.append((v,k))




            # print(character_id, count, max_proper_mention, referential_gender)
            character_data[character_id] = {"id": character_id,
                                  "count": count,
                                  "max_proper_mention": max_proper_mention,
                                  "referential_gender": referential_gender,
                                  "possList": poss_items,
                                  "agentList": agent_items,
                                  "patientList": patient_items,
                                  "modList": mod_items
                                 }
    return character_data
c = create_character_data(data,20)
c

{7920: {'id': 7920,
  'count': 1558,
  'max_proper_mention': 'God',
  'referential_gender': 'he/him/his',
  'possList': [(16, 'name'),
   (11, 'grace'),
   (10, 'family'),
   (10, 'mother'),
   (9, 'hands'),
   (8, 'sister'),
   (8, 'heart'),
   (8, 'love'),
   (7, 'plan'),
   (7, 'life'),
   (6, 'word'),
   (6, 'friend'),
   (6, 'hand'),
   (5, 'help'),
   (5, 'Word'),
   (5, 'people'),
   (5, 'son'),
   (5, 'parents'),
   (5, 'mom'),
   (5, 'baby')],
  'agentList': [(130, 'bless'),
   (17, 'want'),
   (16, 'had'),
   (15, 'know'),
   (14, 'given'),
   (12, 'thank'),
   (12, 'provide'),
   (12, 'do'),
   (11, 'have'),
   (11, 'called'),
   (11, 'blessed'),
   (11, 'Bless'),
   (10, 'asking'),
   (9, 'has'),
   (9, 'keep'),
   (8, 'put'),
   (8, 'get'),
   (7, 'move'),
   (7, 'help'),
   (6, 'give')],
  'patientList': [(18, 'thank'),
   (15, 'Thank'),
   (7, 'help'),
   (4, 'praise'),
   (4, 'trusting'),
   (3, 'join'),
   (3, 'ask'),
   (3, 'know'),
   (2, 'see'),
   (2, 'worship'),
 

In [11]:
character_count = len(c)

In [12]:
df_tokens = pd.read_csv("outputdir/output.tokens", delimiter="\t", error_bad_lines=False)
event_count = 0
for i in df_tokens["event"]:
  if i == "EVENT":
    event_count += 1
event_count

b'Skipping line 39196: expected 13 fields, saw 14\nSkipping line 43513: expected 13 fields, saw 17\nSkipping line 54099: expected 13 fields, saw 14\n'
b'Skipping line 74104: expected 13 fields, saw 15\nSkipping line 91325: expected 13 fields, saw 14\nSkipping line 99663: expected 13 fields, saw 14\nSkipping line 117929: expected 13 fields, saw 23\n'
b'Skipping line 139478: expected 13 fields, saw 14\nSkipping line 145733: expected 13 fields, saw 14\nSkipping line 152233: expected 13 fields, saw 14\nSkipping line 155617: expected 13 fields, saw 17\nSkipping line 182679: expected 13 fields, saw 14\nSkipping line 192848: expected 13 fields, saw 15\nSkipping line 193049: expected 13 fields, saw 14\n'
b'Skipping line 214077: expected 13 fields, saw 15\nSkipping line 226683: expected 13 fields, saw 15\nSkipping line 242651: expected 13 fields, saw 15\nSkipping line 254184: expected 13 fields, saw 15\n'
b'Skipping line 269655: expected 13 fields, saw 14\nSkipping line 279619: expected 13 fiel

24619

In [13]:
action_count = 0

for i in c:
  if c[i]["agentList"] != []:
    action_count += c[i]["agentList"][0][0]
for i in c:
  if c[i]["patientList"] != []:
    action_count += c[i]["patientList"][0][0]
action_count

8780

In [14]:
actions_per_character = action_count / character_count
actions_per_character

1.6494458012399023

In [15]:
import numpy as np
actions_per_character_array = []
for i in c:
  x = 0
  if c[i]["agentList"] != []:
    x += c[i]["agentList"][0][0]
  if c[i]["patientList"]:
    x += c[i]["patientList"][0][0]
  actions_per_character_array.append(x)
actions_per_character_variance = np.var(actions_per_character_array)

In [16]:
female_sentences = 0
male_sentences = 0

for i in c:
  if c[i]["agentList"] != [] and c[i]["referential_gender"] == "she/her":
    male_sentences += c[i]["agentList"][0][0]
  if c[i]["patientList"] and c[i]["referential_gender"] == "she/her":
    male_sentences += c[i]["patientList"][0][0]

for i in c:
  if c[i]["agentList"] != [] and c[i]["referential_gender"] == "he/him/his":
    female_sentences += c[i]["agentList"][0][0]
  if c[i]["patientList"] and c[i]["referential_gender"] == "he/him/his":
    female_sentences += c[i]["patientList"][0][0]
print(female_sentences)
print(male_sentences)

4957
3469


In [17]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

femaleValenceList = []
maleValenceList = []

maleSentScores = []
femaleSentScores = []

for i in c:
  if c[i]["agentList"] != [] and c[i]["referential_gender"] == "she/her":
    val = c[i]["agentList"]
    for j in val:
          femaleValenceList.append(j[1])
  if c[i]["agentList"] != [] and c[i]["referential_gender"] == "he/him/his":
    val = c[i]["agentList"]
    for j in val:
          maleValenceList.append(j[1])
  if c[i]["patientList"] != [] and c[i]["referential_gender"] == "she/her":
    val = c[i]["patientList"]
    for j in val:
          femaleValenceList.append(j[1])
  if c[i]["patientList"] != [] and c[i]["referential_gender"] == "he/him/his":
    val = c[i]["patientList"]
    for j in val:
          maleValenceList.append(j[1])
for text in maleValenceList:
    scores = sid.polarity_scores(text)
    maleSentScores.append(scores['compound'])
for text in femaleValenceList:
    scores = sid.polarity_scores(text)
    femaleSentScores.append(scores['compound'])
averageMaleValence = sum(maleSentScores) / len(maleSentScores)
averageFemaleValence = sum(femaleSentScores) / len(femaleSentScores)
totalAverageValence = averageMaleValence + averageFemaleValence / 2
print(averageMaleValence)
print(averageFemaleValence)
print(totalAverageValence)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gutman.j/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


0.027993695121146564
0.024372013410976133
0.040179701826634634


In [18]:
import os

def clear_folder_contents(folder_path):
    # Check if the folder exists
    if os.path.exists(folder_path):
        # Get the list of files in the folder
        files = os.listdir(folder_path)
        
        # Iterate through the files and remove them
        for file in files:
            file_path = os.path.join(folder_path, file)
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    # Recursively remove subdirectories
                    clear_folder_contents(file_path)
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")
    else:
        print(f"The folder {folder_path} does not exist.")

In [19]:
new_df = pd.DataFrame(columns=['character_count', 'action_count', 'actions_per_character', 'actions_per_character_variance', 'female_sentences', 'male_sentences', 'averageMaleValence', 'averageFemaleValence', 'totalAverageValence'])

In [20]:
len(df)

751118

In [21]:
len(df2)

157868

In [22]:
def find_sentence(text,target_word):
    doc = nlp(text)
    target_sentences = [sent.text for sent in doc.sents if target_word in sent.text.lower()]
    return(target_sentences)

In [23]:
#Bookmark index in case of connectivity issue
start_index = 57184
end_index = 60000
subset_df = df2.iloc[start_index:end_index]
for index, row in subset_df.iterrows():
    char_df = pd.DataFrame(columns=['character_name', 'gender', 'patients','agents' 'sentences', 'file_name'])
    #Prepare data
    clear_folder_contents('outputdirnew-original3')
    new_df = pd.DataFrame(columns=['character_count', 'action_count', 'actions_per_character', 'actions_per_character_variance', 'female_sentences', 'male_sentences', 'averageMaleValence', 'averageFemaleValence', 'totalAverageValence', 'text'])
    text = df2['desc'][index]
    with open('newoutput-original3.txt', 'w') as file:
        file.write(df2['desc'][index])
    get_input('newoutput-original3.txt')
    print(index)
    data=proc("outputdirnew-original3/output.book")
    with open ("outputdirnew-original3/output.book", "r") as f:
        book_data = json.load(f)
    
    c = create_character_data(data,20)
    #Get character count
    character_count = len(c)
    
    try:
    
        df_tokens = pd.read_csv("outputdirnew-original3/output.tokens", delimiter="\t", error_bad_lines=False)
        event_count = 0
        for i in df_tokens["event"]:
          if i == "EVENT":
            event_count += 1
    except pd.errors.ParserError as e:
            print(index)
            continue  # Skip to the next iteration
    #Get action count
    action_count = 0
    
    check_for_dup = {}
    for i in c:
        #Character csv data
        sentences = []
        for j in c[i]['agentList']:
            sentences.append(find_sentence(df2['desc'][index], j[1]))
        for j in c[i]['patientList']:
            sentences.append(find_sentence(df2['desc'][index], j[1]))
        if c[i]["agentList"] != []:
            action_count += c[i]["agentList"][0][0]
        if c[i]['max_proper_mention'][0] not in check_for_dup:
            char_df = char_df.append({'character_name': c[i]['max_proper_mention'],
                                    'gender': c[i]['referential_gender'],
                                    'patients': c[i]['patientList'],
                                    'agents': c[i]['agentList'],
                                    'sentences': sentences,
                                    'file_name': df2['filename'][index]}, ignore_index=True)
            check_for_dup[c[i]['max_proper_mention'][0]] = j
    for i in c:
      if c[i]["patientList"] != []:
        action_count += c[i]["patientList"][0][0]
        
    #Get actions per character
    if character_count > 0:
        actions_per_character = action_count / character_count
    #Get actions per character variance
    actions_per_character_array = []
    for i in c:
      x = 0
      if c[i]["agentList"] != []:
        x += c[i]["agentList"][0][0]
      if c[i]["patientList"]:
        x += c[i]["patientList"][0][0]
      actions_per_character_array.append(x)
    actions_per_character_variance = np.var(actions_per_character_array)
    #Get sentences by gender
    female_sentences = 0
    male_sentences = 0

    for i in c:
      if c[i]["agentList"] != [] and c[i]["referential_gender"] == "she/her":
        male_sentences += c[i]["agentList"][0][0]
      if c[i]["patientList"] and c[i]["referential_gender"] == "she/her":
        male_sentences += c[i]["patientList"][0][0]

    for i in c:
      if c[i]["agentList"] != [] and c[i]["referential_gender"] == "he/him/his":
        female_sentences += c[i]["agentList"][0][0]
      if c[i]["patientList"] and c[i]["referential_gender"] == "he/him/his":
        female_sentences += c[i]["patientList"][0][0]
    #Get valence by gender
    sid = SentimentIntensityAnalyzer()

    femaleValenceList = []
    maleValenceList = []

    maleSentScores = []
    femaleSentScores = []

    for i in c:
      if c[i]["agentList"] != [] and c[i]["referential_gender"] == "she/her":
        val = c[i]["agentList"]
        for j in val:
              femaleValenceList.append(j[1])
      if c[i]["agentList"] != [] and c[i]["referential_gender"] == "he/him/his":
        val = c[i]["agentList"]
        for j in val:
              maleValenceList.append(j[1])
      if c[i]["patientList"] != [] and c[i]["referential_gender"] == "she/her":
        val = c[i]["patientList"]
        for j in val:
              femaleValenceList.append(j[1])
      if c[i]["patientList"] != [] and c[i]["referential_gender"] == "he/him/his":
        val = c[i]["patientList"]
        for j in val:
            maleValenceList.append(find_sentence(df2['desc'][index],j[1]))
    for t in maleValenceList:
        if len(t) > 0:
            scores = sid.polarity_scores(t[0])
            maleSentScores.append(scores['compound'])
    for text in femaleValenceList:
        scores = sid.polarity_scores(text)
        femaleSentScores.append(scores['compound'])
    if len(maleSentScores) > 0:
        averageMaleValence = sum(maleSentScores) / len(maleSentScores)
    if len(femaleSentScores) > 0: 
        averageFemaleValence = sum(femaleSentScores) / len(femaleSentScores)
    totalAverageValence = averageMaleValence + averageFemaleValence / 2
    #Create and append CSV
    new_df = pd.DataFrame(columns=['character_count', 'action_count', 'actions_per_character', 'actions_per_character_variance', 'female_sentences', 'male_sentences', 'averageMaleValence', 'averageFemaleValence', 'totalAverageValence'])
    new_df = new_df.append({'text': df2['desc'][index],
                            'character_count': character_count,
                            'action_count': action_count,
                            'actions_per_character': actions_per_character,
                            'actions_per_character_variance': actions_per_character_variance,
                            'female_sentences': female_sentences,
                            'male_sentences': male_sentences,
                            'averageMaleValence': averageMaleValence,
                            'averageFemaleValence': averageFemaleValence,
                            'totalAverageValence': totalAverageValence}, ignore_index=True)
    new_df.to_csv('df_new3.csv', mode='a', header=None)
    char_df.to_csv('char_df.csv', mode='a', header=None)

--- spacy: 0.055 seconds ---
--- entities: 0.562 seconds ---
--- quotes: 0.000 seconds ---
--- attribution: 0.000 seconds ---
--- name coref: 0.000 seconds ---
--- coref: 3.952 seconds ---
--- TOTAL (excl. startup): 4.604 seconds ---, 110 words
57184


/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--- spacy: 0.114 seconds ---


KeyboardInterrupt: 

In [25]:
df = pd.read_csv('df_new3_with_headers.csv', skiprows=[], error_bad_lines=False)
len(df)

b'Skipping line 467856: expected 11 fields, saw 21\nSkipping line 469681: expected 11 fields, saw 14\nSkipping line 479153: expected 11 fields, saw 30\n'
/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


489887

In [26]:
df_files = {}
for index, row in df.iterrows():
    df_files[df['text'][index]] = index

In [27]:
df.drop_duplicates(inplace=True)

In [28]:
len(df)

231341

In [29]:
df.to_csv('test_csv.csv')

In [33]:
c_df = pd.read_csv('char_df.csv',error_bad_lines=False)

b'Skipping line 26595: expected 8 fields, saw 23\n'
b'Skipping line 68226: expected 8 fields, saw 25\nSkipping line 89290: expected 8 fields, saw 12\nSkipping line 91514: expected 8 fields, saw 31\nSkipping line 128682: expected 8 fields, saw 13\n'
b'Skipping line 168393: expected 8 fields, saw 44\nSkipping line 175195: expected 8 fields, saw 16\nSkipping line 189727: expected 8 fields, saw 11\n'
b'Skipping line 207068: expected 8 fields, saw 11\nSkipping line 207443: expected 8 fields, saw 12\nSkipping line 209004: expected 8 fields, saw 34\nSkipping line 210405: expected 8 fields, saw 14\nSkipping line 213347: expected 8 fields, saw 34\nSkipping line 217549: expected 8 fields, saw 34\nSkipping line 224574: expected 8 fields, saw 9\nSkipping line 225175: expected 8 fields, saw 12\nSkipping line 225393: expected 8 fields, saw 11\nSkipping line 225935: expected 8 fields, saw 22\nSkipping line 228566: expected 8 fields, saw 23\nSkipping line 231097: expected 8 fields, saw 13\nSkipping li

In [35]:
c_df.drop_duplicates(inplace=True)

In [37]:
len(c_df)

194959

In [38]:
c_df.to_csv('final_character_df.csv')

In [36]:
len(char_df)

0

In [ ]:
incidents = []
start_index = 140602
subset_df = df2.iloc[start_index:]
for index, row in subset_df.iterrows():
    if subset_df['desc'][index] not in df_files:
        char_df = pd.DataFrame(columns=['character_name', 'gender', 'patients','agents' 'sentences', 'file_name'])
        #Prepare data
        clear_folder_contents('outputdirnew-original3')
        new_df = pd.DataFrame(columns=['character_count', 'action_count', 'actions_per_character', 'actions_per_character_variance', 'female_sentences', 'male_sentences', 'averageMaleValence', 'averageFemaleValence', 'totalAverageValence', 'text'])
        text = df2['desc'][index]
        with open('newoutput-original3.txt', 'w') as file:
            file.write(df2['desc'][index])
        try:
            get_input('newoutput-original3.txt')
        except Exception as e:
        # Handle other types of exceptions if needed
            print(f"Unexpected exception occurred: {e}")
            continue  # Skip to the next iteration
        print(index)
        data=proc("outputdirnew-original3/output.book")
        with open ("outputdirnew-original3/output.book", "r") as f:
            book_data = json.load(f)

        c = create_character_data(data,20)
        #Get character count
        character_count = len(c)

        try:

            df_tokens = pd.read_csv("outputdirnew-original3/output.tokens", delimiter="\t", error_bad_lines=False)
            event_count = 0
            for i in df_tokens["event"]:
              if i == "EVENT":
                event_count += 1
        except pd.errors.ParserError as e:
                print(index)
                continue  # Skip to the next iteration
        #Get action count
        action_count = 0

        check_for_dup = {}
        for i in c:
            #Character csv data
            sentences = []
            for j in c[i]['agentList']:
                sentences.append(find_sentence(df2['desc'][index], j[1]))
            for j in c[i]['patientList']:
                sentences.append(find_sentence(df2['desc'][index], j[1]))
            if c[i]["agentList"] != []:
                action_count += c[i]["agentList"][0][0]
            if c[i]['max_proper_mention'][0] not in check_for_dup:
                char_df = char_df.append({'character_name': c[i]['max_proper_mention'],
                                        'gender': c[i]['referential_gender'],
                                        'patients': c[i]['patientList'],
                                        'agents': c[i]['agentList'],
                                        'sentences': sentences,
                                        'file_name': df2['filename'][index]}, ignore_index=True)
                check_for_dup[c[i]['max_proper_mention'][0]] = j
        for i in c:
          if c[i]["patientList"] != []:
            action_count += c[i]["patientList"][0][0]

        #Get actions per character
        if character_count > 0:
            actions_per_character = action_count / character_count
        #Get actions per character variance
        actions_per_character_array = []
        for i in c:
          x = 0
          if c[i]["agentList"] != []:
            x += c[i]["agentList"][0][0]
          if c[i]["patientList"]:
            x += c[i]["patientList"][0][0]
          actions_per_character_array.append(x)
        actions_per_character_variance = np.var(actions_per_character_array)
        #Get sentences by gender
        female_sentences = 0
        male_sentences = 0

        for i in c:
          if c[i]["agentList"] != [] and c[i]["referential_gender"] == "she/her":
            male_sentences += c[i]["agentList"][0][0]
          if c[i]["patientList"] and c[i]["referential_gender"] == "she/her":
            male_sentences += c[i]["patientList"][0][0]

        for i in c:
          if c[i]["agentList"] != [] and c[i]["referential_gender"] == "he/him/his":
            female_sentences += c[i]["agentList"][0][0]
          if c[i]["patientList"] and c[i]["referential_gender"] == "he/him/his":
            female_sentences += c[i]["patientList"][0][0]
        #Get valence by gender
        sid = SentimentIntensityAnalyzer()

        femaleValenceList = []
        maleValenceList = []

        maleSentScores = []
        femaleSentScores = []

        for i in c:
          if c[i]["agentList"] != [] and c[i]["referential_gender"] == "she/her":
            val = c[i]["agentList"]
            for j in val:
                  femaleValenceList.append(j[1])
          if c[i]["agentList"] != [] and c[i]["referential_gender"] == "he/him/his":
            val = c[i]["agentList"]
            for j in val:
                  maleValenceList.append(j[1])
          if c[i]["patientList"] != [] and c[i]["referential_gender"] == "she/her":
            val = c[i]["patientList"]
            for j in val:
                  femaleValenceList.append(j[1])
          if c[i]["patientList"] != [] and c[i]["referential_gender"] == "he/him/his":
            val = c[i]["patientList"]
            for j in val:
                maleValenceList.append(find_sentence(df2['desc'][index],j[1]))
        for t in maleValenceList:
            if len(t) > 0:
                scores = sid.polarity_scores(t[0])
                maleSentScores.append(scores['compound'])
        for t in femaleValenceList:
            if len(t) > 0:
                scores = sid.polarity_scores(t[0])
                femaleSentScores.append(scores['compound'])
        if len(maleSentScores) > 0:
            averageMaleValence = sum(maleSentScores) / len(maleSentScores)
        if len(femaleSentScores) > 0: 
            averageFemaleValence = sum(femaleSentScores) / len(femaleSentScores)
        totalAverageValence = averageMaleValence + averageFemaleValence / 2
        #Create and append CSV
        new_df = pd.DataFrame(columns=['character_count', 'action_count', 'actions_per_character', 'actions_per_character_variance', 'female_sentences', 'male_sentences', 'averageMaleValence', 'averageFemaleValence', 'totalAverageValence'])
        new_df = new_df.append({'text': df2['desc'][index],
                                'character_count': character_count,
                                'action_count': action_count,
                                'actions_per_character': actions_per_character,
                                'actions_per_character_variance': actions_per_character_variance,
                                'female_sentences': female_sentences,
                                'male_sentences': male_sentences,
                                'averageMaleValence': averageMaleValence,
                                'averageFemaleValence': averageFemaleValence,
                                'totalAverageValence': totalAverageValence}, ignore_index=True)
        new_df.to_csv('df_new3_with_headers.csv', mode='a', header=None)
        char_df.to_csv('char_df.csv', mode='a', header=None)


In [ ]:
print('clear')

In [ ]:
df2['desc'][2]

In [54]:
df3 = pd.read_csv('df_new2.csv')
df3

,Unnamed: 0,character_count,action_count,actions_per_character,actions_per_character_variance,female_sentences,male_sentences,averageMaleValence,averageFemaleValence,totalAverageValence,text
0,0.0,1,1,1.0,0.0,1,0,0.0,-0.3182,-0.1591,leaves
1,NaN,character_count,action_count,actions_per_character,actions_per_character_variance,female_sentences,male_sentences,averageMaleValence,averageFemaleValence,totalAverageValence,text
2,0.0,0,0,1.0,NaN,0,0,0.0,-0.3182,-0.1591,"Hello I'm Katherine, and I need help with my r..."
3,NaN,character_count,action_count,actions_per_character,actions_per_character_variance,female_sentences,male_sentences,averageMaleValence,averageFemaleValence,totalAverageValence,text
4,0.0,1,1,1.0,0.0,1,0,0.0,-0.3182,-0.1591,leaves
...,...,...,...,...,...,...,...,...,...,...,...
1156,0.0,1,1,1.0,0.0,1,0,0.0,0.0,0.0,walked
1157,NaN,character_count,action_count,actions_per_character,actions_per_character_variance,female_sentences,male_sentences,averageMaleValence,averageFemaleValence,totalAverageValence,text
1158,0.0,1,1,1.0,0.0,0,1,0.0,0.0,0.0,going
1159,NaN,character_count,action_count,actions_per_character,actions_per_character_variance,female_sentences,male_sentences,averageMaleValence,averageFemaleValence,totalAverageValence,text


In [17]:
for i in range(1):
    print("clear")

clear


AttributeError: 'DataFrame' object has no attribute 'rows'